In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from fake_useragent import UserAgent
import datetime
import time
import random
from sqlalchemy import create_engine
import mysql.connector


In [4]:
ua = UserAgent()
headers = {"User-Agent":ua.random}

In [5]:
url="https://isin.twse.com.tw/isin/C_public.jsp?strMode=2"

def Get_stock(url):
    res = requests.get(url,headers=headers)
    soup = BeautifulSoup(res.text,'html.parser')
    table = soup.find("table",class_="h4")
    stock_list = []

    for tr in table.find_all("tr")[2:]:
        try:
            storage = tr.find("td").text.split("\u3000",1)
            stock_number = int(storage[0])
            stock_name = storage[1]

            stock_createdate = tr.find_all("td")[2].text

            stock_class = tr.find_all("td")[3].text
            stock_type = tr.find_all("td")[4].text

            stock_dict = {
                "stock_number":stock_number,
                "stock_name":stock_name,
                "stock_createdate":stock_createdate,
                "stock_class":stock_class,
                "stock_type":stock_type
            }

            stock_list.append(stock_dict)

        except IndexError:
            pass
        except ValueError:
            break
            
    return stock_list
    
stock_list = Get_stock(url)

In [6]:
#連結資料庫
#範本解釋:create_engine('mysql+mysql_driver://mysql帳號:mysql密碼@機器ip:mysql_port/DB名稱?其他參數', encoding='mysql編碼'
#charset=utf8 資料庫編碼

user = "root"
passwd = "Pn123456"
ip = "192.168.22.112:3306"
db_name = "Stock"
table_name = "Stock_information"

engine = create_engine('mysql+mysqlconnector://'+ user +':'+ passwd +'@'+ip+'/'+ db_name +'?charset=utf8', encoding='utf-8')
con = engine.connect() #建立連結

for item in stock_list:
    df = pd.DataFrame(item, index=[0])
    try:
        df.to_sql(name=table_name,con=con,if_exists='append',index=False) #假設table已存在 就自動往下加入data
        
    except Exception as e:
        if 'PRIMARY' in str(e):
            pass
        
con.close() 
engine.dispose()

InterfaceError: (mysql.connector.errors.InterfaceError) 2003: Can't connect to MySQL server on '192.168.56.112:3306' (10060 連線嘗試失敗，因為連線對象有一段時間並未正確回應，或是連線建立失敗，因為連線的主機無法回應。)
(Background on this error at: http://sqlalche.me/e/rvf5)

In [ ]:
stock_list